In [2]:
import cv2
import matplotlib.pyplot as plt
import numpy as np

kernel = np.matrix('0 -1 0 ; -1 5 -1 ; 0 -1 0') 
img = cv2.imread('bowler.png',0)
frame = cv2.filter2D(img,-1,kernel)
#frame = frame[220:520,450:900]
score = img[620:650,290:390]
#plt.imshow(score,cmap='gray')
umpire = img[300:530,620:725]
hist = cv2.calcHist([umpire],[0],None,[256],[0,256])
#plt.imshow(umpire,cmap='gray')
mask = cv2.calcBackProject([frame],[0,1],hist,[0,180,0,256],1)
disc = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(5,5))
mask = cv2.filter2D(mask,-1,disc)
#plt.imshow(frame,cmap='gray')
kernel = np.ones((3,3)) 
mask = cv2.erode(mask, kernel, iterations = 3)
mask = cv2.dilate(mask, kernel, iterations = 1)
#plt.imshow(score,cmap='gray')
#plt.imshow(score,cmap='gray')

In [3]:
ret,test = cv2.threshold(umpire,125,255,1)
test = test[0:230,:]
test = cv2.dilate(test, kernel, iterations = 1)
test = cv2.erode(test, kernel, iterations = 1)

#plt.imshow(test,cmap='gray')

In [4]:
a = np.ones((10,30,16))
names = ['0','1','2','3','4','5','6','7','8','9']
for i in  range(10):
    a[i] = cv2.imread('nums/'+names[i]+'.png',0)
    ret,a[i] = cv2.threshold(a[i],100,255,cv2.THRESH_BINARY)
#plt.imshow(a[5],cmap='gray')

In [18]:
def hl_det(c,p):
    if(c[2]==1000):
        cs = c[0]
        cw = c[1]
    else : 
        if (c[3]==1000):
            cs = 10*c[0] + c[1]
            cw = c[2]
        else:
            cs = 100*c[0]+10*c[1]+c[2]
            cw = c[3]
    if(p[2]==1000):
        ps = p[0]
        pw = p[1]
    else : 
        if (p[3]==1000):
            ps = 10*p[0] + p[1]
            pw = p[2]
        else :
            ps = 100*p[0]+10*p[1]+p[2]
            pw = p[3]
    
    #print(cs,ps,cw,pw)
    sco = cs - ps
    wic = cw - pw
    
    return int(sco),int(wic)

In [26]:
vid = cv2.VideoCapture('cric_hl.mp4')
min = 65
sec = 0
#frame_no = (60*min +sec)*25
frame_no = 102211
brk = 0;
break_con = 0;
pre_score = np.ones(4)*2000
cur_score = np.ones(4)*1000
vid.set(1,frame_no);
pre_frame = frame_no

while(vid.isOpened()):
    ret, frame =vid.read()
    kernel = np.matrix('0 -1 0 ; -1 5 -1 ; 0 -1 0') 
    #score0 = frame0[620:650,290:390]
    frame0 = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    #frame0 = cv2.filter2D(frame0,-1,kernel)
    #frame0 = frame0[220:520,450:900]
    ret,calc = cv2.threshold(frame0,100,255,cv2.THRESH_BINARY)
    score_t = calc[620:650,295:385]
    
    
    mask = cv2.calcBackProject([frame0],[0,1],hist,[0,180,0,256],1)
    mask = cv2.filter2D(mask,-1,disc)
    ret, mask=cv2.threshold(mask,75,255,0)
    kernel = np.ones((3,3)) 
    mask = cv2.erode(mask, kernel, iterations = 1)
    mask = cv2.dilate(mask, kernel, iterations = 1)
    #mask = cv2.merge((mask,mask,mask))
    result=cv2.bitwise_and(frame0,mask)
    cv2.imshow('vid',score_t)
    #cv2.imshow('vid',mask)
    minm = 5000000
    
    if break_con ==1 :
        brk = brk + 1;
        if brk == 50:
            brk = 0
            break_con = 0
    
    if brk == 0 :
        for x in range(300,371,10):
            for y in range(500,596,10):
                temp = mask[x:x+230,y:y+105]
                if minm>np.absolute(np.sum(temp-test)):
                    minm = np.sum(np.absolute(temp-test))
        
        change = 0
        index = 0
        if minm<500000:
            #print('$:',pre_score)
            break_con = 1;
            pi = -10
            for i in range(75):
                temp = score_t[:,i:i+16]
                minm = 10000
                #print('*')
                for j in range(10):
                    if minm>np.sum(np.absolute(temp - a[j])):
                        if (i-pi) > 10 :
                            cur_score[index]=j
                            index = index + 1
                            pi = i
                            change = 1
            #print(not((cur_score==pre_score).all()),frame_no,cur_score,pre_score)
            if (change ==1):
                if (not((cur_score==pre_score).all())):
                    hl,wic = hl_det(cur_score, pre_score)
                    print (hl,wic, 'from:',pre_frame,'  to:',frame_no)
                    pre_score = cur_score.copy()
                    pre_frame = frame_no
                else:
                    pre_frame = frame_no
            
            #cv2.imshow('frame',frame)
            #cv2.waitKey(0)
            #cv2.destroyAllWindows()
            #print(cur_score,pre_score)
    change = 0
    frame_no = frame_no + 1
    if cv2.waitKey(25) & 0xFF == ord('q'):
      break
vid.release()
cv2.destroyAllWindows()

-221902 -1996 from: 102211   to: 103330
1 0 from: 103909   to: 104931
1 0 from: 105825   to: 107862
4 0 from: 107862   to: 108986
0 1 from: 109211   to: 111396
1 0 from: 111909   to: 113217
4 0 from: 113217   to: 114266
1 0 from: 115314   to: 116947
1 0 from: 116947   to: 117125
7 0 from: 117873   to: 119163
1 0 from: 119641   to: 119758
1 0 from: 120375   to: 120523
6 0 from: 122393   to: 123435
6 0 from: 124352   to: 125390
1 0 from: 125440   to: 125777
0 1 from: 127686   to: 127912
1 0 from: 129633   to: 131473
1 0 from: 131523   to: 132534
1 0 from: 132534   to: 132832


In [31]:
hl = cv2.VideoCapture('cric_hl.mp4')
frame_ini = 109211-25
frame_fin = 111396
frame_no = frame_ini
hl.set(1,frame_ini)
while(hl.isOpened()):
    ret, frame =hl.read()
    cv2.imshow('highlight',frame)
    frame_no = frame_no + 1
    
    if (frame_no == frame_fin):
      break
    
    if cv2.waitKey(25) & 0xFF == ord('q'):
      break

hl.release()
cv2.destroyAllWindows()